In [1]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings
import yfinance as yf
import langchain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

warnings.filterwarnings("ignore")

In [2]:
llm = ChatOpenAI(temperature=0.5, verbose=True)

In [3]:
#All Stock Info
yf.Ticker("RVNL.NS").info

{'address1': 'August Kranti Bhawan',
 'address2': 'First Floor Plot No. 25, Bhikaji Cama Place R. K. Puram',
 'city': 'New Delhi',
 'zip': '110066',
 'country': 'India',
 'phone': '91 11 2673 8299',
 'fax': '91 11 2618 2957',
 'website': 'https://www.rvnl.org',
 'industry': 'Engineering & Construction',
 'industryKey': 'engineering-construction',
 'industryDisp': 'Engineering & Construction',
 'sector': 'Industrials',
 'sectorKey': 'industrials',
 'sectorDisp': 'Industrials',
 'longBusinessSummary': 'Rail Vikas Nigam Limited engages in the construction of rail infrastructure projects in India. It undertakes rail project development and implementation, as well as provides financial resources mobilization services. The company executes various railway projects, including new lines, doubling, gauge conversion, railway electrification, workshops, bridges, metro projects, construction of cable stayed bridges, institution buildings, etc. It serves Indian Railways, various central and state g

In [4]:
yf.Ticker("RVNL.NS").history(period="1mo")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-03-10 00:00:00+05:30,343.000000,349.000000,333.950012,335.700012,6498796,0.0,0.0
2025-03-11 00:00:00+05:30,326.049988,333.450012,322.049988,330.649994,8033534,0.0,0.0
2025-03-12 00:00:00+05:30,340.000000,342.000000,329.000000,333.299988,8658207,0.0,0.0
2025-03-13 00:00:00+05:30,336.000000,336.350006,327.549988,329.350006,5491120,0.0,0.0
2025-03-17 00:00:00+05:30,330.500000,334.200012,325.350006,328.450012,5347960,0.0,0.0
2025-03-18 00:00:00+05:30,328.450012,328.450012,328.450012,328.450012,0,0.0,0.0
2025-03-19 00:00:00+05:30,334.600006,361.950012,334.000000,353.649994,22821539,0.0,0.0
2025-03-20 00:00:00+05:30,357.500000,366.750000,351.500000,357.899994,13396119,0.0,0.0
2025-03-21 00:00:00+05:30,360.200012,374.000000,352.549988,361.350006,17632927,0.0,0.0


In [28]:

def get_stock_price(ticker,history=5):
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df=df[["Open","Close","Volume"]]
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df=df[-history:]
    # print(df.columns)
    
    return df.to_string()

print(get_stock_price("RVNL"))

                  Open       Close   Volume
Date                                       
2025-04-03  351.000000  359.750000  6036464
2025-04-04  357.950012  350.299988  5639050
2025-04-07  305.000000  335.950012  8638161
2025-04-08  347.850006  344.350006  4587642
2025-04-09  343.850006  337.649994  3740535


In [14]:
# Script to scrap top5 googgle news for given company name

def google_query(search_term):
    if "news" not in search_term:
        search_term = search_term + " stock news"
    
    # Modified URL to access Google News tab directly
    url = f"https://www.google.com/search?q={search_term}&tbm=nws&cr=countryIN"
    url = re.sub(r"\s", "+", url)
    return url



def get_recent_stock_news(company_name):
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    print
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","n0jPhd ynAwRc MBeuO nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)
    print(news)

    if len(news)>6:
        news=news[:5]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i+1}. {n}\n"
    top5_news="Recent News:\n\n"+news_string
    
    return top5_news


print(get_recent_stock_news("RVNL"))

['IRFC, RVNL to RailTel: Why will railway PSU stocks be in focus on Monday? EXPLAINED', 'Rail Vikas Nigam Limited (RVNL) Share Price in Focus; Immediate Target at Rs 368', 'RVNL stock falls 42% from record high; value buy or a falling comet?', 'RVNL’s share price jumped 3% on securing ₹156 crore order from South Western Railway', "Stocks to buy: Swiggy, RVNL and TVS Motor on investors' radar", 'RVNL gains 3% on emerging L1 bidder for ₹115-cr order from Central Railway', 'RVNL share price gains on fresh order win from Central Railway. Do you own?', "RVNL stock turned into a multibagger since Coronavirus crash, here's what analysts say", 'RVNL share price up 8% on bagging Rs 729-cr order from HP State Electricity', 'RVNL shares rally 3% after winning Rs 116-crore Central Railway contract in Nagpur']
Recent News:

1. IRFC, RVNL to RailTel: Why will railway PSU stocks be in focus on Monday? EXPLAINED
2. Rail Vikas Nigam Limited (RVNL) Share Price in Focus; Immediate Target at Rs 368
3. RVN

In [ ]:
# # Fetch financial statements from Yahoo Finance

# def get_financial_statements(ticker):
#     if "." in ticker:
#         ticker=ticker.split(".")[0]
#     else:
#         ticker=ticker
#     ticker=ticker+".NS"    
#     company = yf.Ticker(ticker)
#     balance_sheet = company.balance_sheet
#     # print(balance_sheet.shape[0])
#     if balance_sheet.shape[1]>=3:
#         balance_sheet=balance_sheet.iloc[:,:3]    # Remove 4th years data
#     balance_sheet=balance_sheet.dropna(how="any")
#     balance_sheet = balance_sheet.to_string()
    
#     # cash_flow = company.cash_flow.to_string()
#     # print(balance_sheet)
#     # print(cash_flow)
#     return balance_sheet
# print(get_financial_statements("RVNL.NS"))

def get_financial_statements(ticker):
    """
    Retrieves and formats comprehensive financial statements for a given stock ticker.
    Returns well-structured data with key financial metrics highlighted for LLM analysis.
    """
    if "." in ticker:
        ticker = ticker.split(".")[0]
    ticker = ticker + ".NS"
    
    company = yf.Ticker(ticker)
    
    # Get company information
    try:
        company_info = company.info
        company_name = company_info.get('longName', company_info.get('shortName', ticker))
        sector = company_info.get('sector', 'N/A')
        industry = company_info.get('industry', 'N/A')
        summary = company_info.get('longBusinessSummary', '')
    except:
        company_name = ticker
        sector = industry = 'N/A'
        summary = ''
    
    # Get financial statements
    balance_sheet = company.balance_sheet
    income_stmt = company.income_stmt
    cash_flow = company.cashflow
    
    # Format balance sheet (keep up to 3 years of data)
    if balance_sheet.shape[1] >= 3:
        balance_sheet = balance_sheet.iloc[:, :3]
    balance_sheet = balance_sheet.dropna(how="any")
    
    # Start building the formatted output
    output = f"FINANCIAL STATEMENTS ANALYSIS FOR {company_name} ({ticker})\n"
    output += f"Sector: {sector} | Industry: {industry}\n"
    output += "=" * 80 + "\n\n"
    
    # Add brief company description if available
    if summary:
        # Truncate to ~150 words
        if len(summary) > 1000:
            summary = summary[:1000] + "..."
        output += f"COMPANY OVERVIEW:\n{summary}\n\n"
    
    # Add key financial metrics section with improved formatting
    output += "KEY FINANCIAL METRICS AND RATIOS:\n"
    output += "-" * 40 + "\n"
    
    # Extract key metrics from balance sheet
    try:
        # ASSETS AND LIABILITIES
        if 'Total Assets' in balance_sheet.index:
            total_assets = balance_sheet.loc['Total Assets']
            output += f"Total Assets (Latest): ₹{total_assets.iloc[0]/10000000:.2f} Cr"
            
            if len(total_assets) >= 2:
                yoy_asset_growth = ((total_assets.iloc[0] / total_assets.iloc[1]) - 1) * 100
                output += f" | YoY Growth: {yoy_asset_growth:.2f}%\n"
            else:
                output += "\n"
        
        if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
            total_liabilities = balance_sheet.loc['Total Liabilities Net Minority Interest']
            output += f"Total Liabilities (Latest): ₹{total_liabilities.iloc[0]/10000000:.2f} Cr\n"
        
        # EQUITY AND DEBT
        if 'Total Stockholder Equity' in balance_sheet.index:
            equity = balance_sheet.loc['Total Stockholder Equity'].iloc[0]
            output += f"Shareholder Equity: ₹{equity/10000000:.2f} Cr"
            
            if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
                total_liabilities = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
                debt_equity_ratio = total_liabilities / equity
                output += f" | Debt-to-Equity: {debt_equity_ratio:.2f}\n"
            else:
                output += "\n"
                
        # Add more balance sheet ratios
        if 'Total Assets' in balance_sheet.index and 'Total Stockholder Equity' in balance_sheet.index:
            equity_ratio = balance_sheet.loc['Total Stockholder Equity'].iloc[0] / balance_sheet.loc['Total Assets'].iloc[0]
            output += f"Equity Ratio: {equity_ratio:.2f} ({(equity_ratio*100):.1f}% assets financed by equity)\n"
            
        if 'Current Assets' in balance_sheet.index and 'Current Liabilities' in balance_sheet.index:
            current_ratio = balance_sheet.loc['Current Assets'].iloc[0] / balance_sheet.loc['Current Liabilities'].iloc[0]
            output += f"Current Ratio: {current_ratio:.2f}"
            
            if current_ratio > 2:
                output += " (Strong liquidity)\n"
            elif current_ratio > 1:
                output += " (Adequate liquidity)\n"
            else:
                output += " (Potential liquidity concerns)\n"
    except Exception as e:
        output += f"Error extracting balance sheet metrics: {str(e)}\n"
    
    # Add income statement metrics if available
    output += "\nPROFITABILITY METRICS:\n"
    output += "-" * 40 + "\n"
    
    try:
        if not income_stmt.empty:
            # REVENUE AND INCOME
            if 'Total Revenue' in income_stmt.index:
                revenue = income_stmt.loc['Total Revenue'].iloc[0]
                output += f"Revenue (Latest): ₹{revenue/10000000:.2f} Cr"
                
                if len(income_stmt.loc['Total Revenue']) >= 2:
                    prev_revenue = income_stmt.loc['Total Revenue'].iloc[1]
                    rev_growth = ((revenue / prev_revenue) - 1) * 100
                    output += f" | YoY Growth: {rev_growth:.2f}%\n"
                else:
                    output += "\n"
            
            if 'Net Income' in income_stmt.index:
                net_income = income_stmt.loc['Net Income'].iloc[0]
                output += f"Net Income: ₹{net_income/10000000:.2f} Cr"
                
                if 'Total Revenue' in income_stmt.index:
                    revenue = income_stmt.loc['Total Revenue'].iloc[0]
                    profit_margin = (net_income / revenue) * 100
                    output += f" | Profit Margin: {profit_margin:.2f}%\n"
                else:
                    output += "\n"
                    
            # EPS and P/E RATIO
            if 'Basic EPS' in income_stmt.index:
                eps = income_stmt.loc['Basic EPS'].iloc[0]
                output += f"EPS (Basic): ₹{eps:.2f}"
                
                try:
                    # Try to get current stock price
                    current_price = company.info.get('currentPrice', company.history(period="1d")['Close'].iloc[-1])
                    pe_ratio = current_price / eps
                    output += f" | P/E Ratio: {pe_ratio:.2f}\n"
                except:
                    output += "\n"
            
            # EBITDA and MARGINS
            if 'EBITDA' in income_stmt.index:
                ebitda = income_stmt.loc['EBITDA'].iloc[0]
                output += f"EBITDA: ₹{ebitda/10000000:.2f} Cr"
                
                if 'Total Revenue' in income_stmt.index:
                    ebitda_margin = (ebitda / income_stmt.loc['Total Revenue'].iloc[0]) * 100
                    output += f" | EBITDA Margin: {ebitda_margin:.2f}%\n"
                else:
                    output += "\n"
    except Exception as e:
        output += f"Error extracting income statement metrics: {str(e)}\n"
    
    # Add cash flow metrics
    output += "\nCASH FLOW METRICS:\n"
    output += "-" * 40 + "\n"
    
    try:
        if not cash_flow.empty:
            # OPERATING CASH FLOW
            if 'Operating Cash Flow' in cash_flow.index:
                ocf = cash_flow.loc['Operating Cash Flow'].iloc[0]
                output += f"Operating Cash Flow: ₹{ocf/10000000:.2f} Cr\n"
            
            # FREE CASH FLOW
            if 'Free Cash Flow' in cash_flow.index:
                fcf = cash_flow.loc['Free Cash Flow'].iloc[0]
                output += f"Free Cash Flow: ₹{fcf/10000000:.2f} Cr"
                
                if 'Net Income From Continuing Operations' in cash_flow.index:
                    fcf_to_income = (fcf / cash_flow.loc['Net Income From Continuing Operations'].iloc[0]) * 100
                    output += f" | FCF to Net Income: {fcf_to_income:.2f}%\n"
                else:
                    output += "\n"
            
            # CAPEX
            if 'Capital Expenditure Reported' in cash_flow.index:
                capex = cash_flow.loc['Capital Expenditure Reported'].iloc[0]
                output += f"Capital Expenditure: ₹{abs(capex)/10000000:.2f} Cr\n"
    except Exception as e:
        output += f"Error extracting cash flow metrics: {str(e)}\n"
    
    # Add the raw balance sheet data
    output += "\nBALANCE SHEET (Last 3 Years):\n"
    output += "-" * 40 + "\n"
    output += balance_sheet.to_string() + "\n\n"
    
    # Add raw income statement data (last 3 years)
    try:
        if not income_stmt.empty:
            if income_stmt.shape[1] >= 3:
                income_stmt = income_stmt.iloc[:, :3]
            income_stmt = income_stmt.dropna(how="any")
            output += "INCOME STATEMENT (Last 3 Years):\n"
            output += "-" * 40 + "\n"
            output += income_stmt.to_string() + "\n\n"
    except Exception as e:
        output += f"Error processing income statement: {str(e)}\n\n"
    
    # Add raw cash flow data (last 3 years)
    try:
        if not cash_flow.empty:
            if cash_flow.shape[1] >= 3:
                cash_flow = cash_flow.iloc[:, :3]
            cash_flow = cash_flow.dropna(how="any")
            output += "CASH FLOW STATEMENT (Last 3 Years):\n"
            output += "-" * 40 + "\n"
            output += cash_flow.to_string() + "\n\n"
    except Exception as e:
        output += f"Error processing cash flow statement: {str(e)}\n\n"
    
    # Add financial health summary
    output += "FINANCIAL HEALTH SUMMARY:\n"
    output += "-" * 40 + "\n"
    
    try:
        # Calculate financial health assessment
        health_indicators = []
        
        # Equity position
        if 'Total Stockholder Equity' in balance_sheet.index and 'Total Assets' in balance_sheet.index:
            equity_ratio = balance_sheet.loc['Total Stockholder Equity'].iloc[0] / balance_sheet.loc['Total Assets'].iloc[0]
            if equity_ratio > 0.5:
                health_indicators.append("Strong equity position (>50% assets financed by equity)")
            elif equity_ratio > 0.3:
                health_indicators.append("Moderate equity position (30-50% assets financed by equity)")
            else:
                health_indicators.append("Lower equity position (<30% assets financed by equity)")
        
        # Liquidity
        if 'Current Assets' in balance_sheet.index and 'Current Liabilities' in balance_sheet.index:
            current_ratio = balance_sheet.loc['Current Assets'].iloc[0] / balance_sheet.loc['Current Liabilities'].iloc[0]
            if current_ratio > 2:
                health_indicators.append("Strong short-term liquidity (current ratio >2)")
            elif current_ratio > 1:
                health_indicators.append("Adequate short-term liquidity (current ratio >1)")
            else:
                health_indicators.append("Potential short-term liquidity concerns (current ratio <1)")
        
        # Profitability
        if not income_stmt.empty and 'Net Income' in income_stmt.index and 'Total Revenue' in income_stmt.index:
            profit_margin = (income_stmt.loc['Net Income'].iloc[0] / income_stmt.loc['Total Revenue'].iloc[0]) * 100
            if profit_margin > 15:
                health_indicators.append(f"Excellent profitability (profit margin {profit_margin:.1f}%)")
            elif profit_margin > 8:
                health_indicators.append(f"Good profitability (profit margin {profit_margin:.1f}%)")
            elif profit_margin > 3:
                health_indicators.append(f"Moderate profitability (profit margin {profit_margin:.1f}%)")
            else:
                health_indicators.append(f"Lower profitability (profit margin {profit_margin:.1f}%)")
        
        # Cash flow health
        if not cash_flow.empty and 'Free Cash Flow' in cash_flow.index and 'Net Income From Continuing Operations' in cash_flow.index:
            fcf_ratio = cash_flow.loc['Free Cash Flow'].iloc[0] / cash_flow.loc['Net Income From Continuing Operations'].iloc[0]
            if fcf_ratio > 1:
                health_indicators.append(f"Strong cash generation (FCF/Net Income ratio {fcf_ratio:.1f})")
            elif fcf_ratio > 0.5:
                health_indicators.append(f"Adequate cash generation (FCF/Net Income ratio {fcf_ratio:.1f})")
            else:
                health_indicators.append(f"Weak cash generation relative to earnings (FCF/Net Income ratio {fcf_ratio:.1f})")
        
        # Output health indicators
        for i, indicator in enumerate(health_indicators):
            output += f"{i+1}. {indicator}\n"
            
        if not health_indicators:
            output += "Insufficient data to provide detailed financial health assessment.\n"
            
    except Exception as e:
        output += f"Error calculating financial health indicators: {str(e)}\n"
    
    return output
ans = get_financial_statements("RVNL.NS")
ans

with open("RVNL.NS.txt", "w", encoding="utf-8") as f:
    f.write(ans)





: 

In [24]:
def get_stock_info_for_month(ticker):
    return yf.Ticker(ticker).history(period="1mo")

ticker="RVNL.NS"
ans = get_stock_info_for_month(ticker)
ans


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-03-10 00:00:00+05:30,343.000000,349.000000,333.950012,335.700012,6498796,0.0,0.0
2025-03-11 00:00:00+05:30,326.049988,333.450012,322.049988,330.649994,8033534,0.0,0.0
2025-03-12 00:00:00+05:30,340.000000,342.000000,329.000000,333.299988,8658207,0.0,0.0
2025-03-13 00:00:00+05:30,336.000000,336.350006,327.549988,329.350006,5491120,0.0,0.0
2025-03-17 00:00:00+05:30,330.500000,334.200012,325.350006,328.450012,5347960,0.0,0.0
2025-03-18 00:00:00+05:30,328.450012,328.450012,328.450012,328.450012,0,0.0,0.0
2025-03-19 00:00:00+05:30,334.600006,361.950012,334.000000,353.649994,22821539,0.0,0.0
2025-03-20 00:00:00+05:30,357.500000,366.750000,351.500000,357.899994,13396119,0.0,0.0
2025-03-21 00:00:00+05:30,360.200012,374.000000,352.549988,361.350006,17632927,0.0,0.0


In [26]:
def get_stock_info_for_month(ticker):
    """Get monthly stock information formatted for LLM consumption with complete OHLC data"""
    # Get the raw data
    stock_data = yf.Ticker(ticker).history(period="1mo")
    
    # Calculate key metrics
    current_price = stock_data['Close'].iloc[-1]
    start_price = stock_data['Close'].iloc[0]
    price_change = current_price - start_price
    percent_change = (price_change / start_price) * 100
    
    # Overall statistics
    high_price = stock_data['High'].max()
    low_price = stock_data['Low'].min()
    avg_volume = stock_data['Volume'].mean()
    
    # Add company info if available
    try:
        ticker_info = yf.Ticker(ticker).info
        company_name = ticker_info.get('shortName', ticker)
        market_cap = ticker_info.get('marketCap', 'N/A')
        if market_cap != 'N/A':
            market_cap = f"₹{market_cap/10000000:.2f} Cr"
    except:
        company_name = ticker
        market_cap = 'N/A'
    
    # Format the data as a string
    summary = f"""
Monthly Stock Summary for {company_name} ({ticker}):
=====================================
SUMMARY METRICS:
- Current Price: ₹{current_price:.2f}
- Monthly Change: ₹{price_change:.2f} ({percent_change:.2f}%)
- Monthly Range: ₹{low_price:.2f} (Low) to ₹{high_price:.2f} (High)
- Average Daily Volume: {int(avg_volume):,} shares
- Market Cap: {market_cap}

RECENT TRADING DATA (Last 5 days with OHLC):
"""
    
    # Add last 5 days of detailed OHLC data
    for date, row in stock_data[-5:].iterrows():
        date_str = date.strftime('%Y-%m-%d')
        daily_change = ((row['Close'] - row['Open']) / row['Open']) * 100
        change_symbol = "▲" if daily_change >= 0 else "▼"
        
        summary += f"- {date_str}:\n"
        summary += f"  * Open: ₹{row['Open']:.2f}\n"
        summary += f"  * High: ₹{row['High']:.2f}\n"
        summary += f"  * Low: ₹{row['Low']:.2f}\n"
        summary += f"  * Close: ₹{row['Close']:.2f} {change_symbol} {abs(daily_change):.2f}%\n"
        summary += f"  * Volume: {int(row['Volume']):,}\n"
    
    # Add technical indicators
    summary += "\nTECHNICAL INDICATORS:\n"
    
    # 7-day moving average (if we have enough data)
    if len(stock_data) >= 7:
        ma7 = stock_data['Close'].rolling(window=7).mean().iloc[-1]
        summary += f"- 7-Day Moving Average: ₹{ma7:.2f}\n"
    
    # RSI calculation (simplified)
    try:
        delta = stock_data['Close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=14).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs)).iloc[-1]
        summary += f"- 14-Day RSI: {rsi:.2f}\n"
    except:
        pass
    
    return summary

# Example usage
ticker = "RVNL.NS"
stock_info = get_stock_info_for_month(ticker)
print(stock_info)  # Print for notebook display


Monthly Stock Summary for RAIL VIKAS NIGAM LIMITED (RVNL.NS):
SUMMARY METRICS:
- Current Price: ₹337.65
- Monthly Change: ₹1.95 (0.58%)
- Monthly Range: ₹305.00 (Low) to ₹383.95 (High)
- Average Daily Volume: 8,509,036 shares
- Market Cap: ₹70400.70 Cr

RECENT TRADING DATA (Last 5 days with OHLC):
- 2025-04-03:
  * Open: ₹351.00
  * High: ₹365.00
  * Low: ₹351.00
  * Close: ₹359.75 ▲ 2.49%
  * Volume: 6,036,464
- 2025-04-04:
  * Open: ₹357.95
  * High: ₹360.70
  * Low: ₹344.15
  * Close: ₹350.30 ▼ 2.14%
  * Volume: 5,639,050
- 2025-04-07:
  * Open: ₹305.00
  * High: ₹340.00
  * Low: ₹305.00
  * Close: ₹335.95 ▲ 10.15%
  * Volume: 8,638,161
- 2025-04-08:
  * Open: ₹347.85
  * High: ₹348.95
  * Low: ₹338.80
  * Close: ₹344.35 ▼ 1.01%
  * Volume: 4,587,642
- 2025-04-09:
  * Open: ₹343.85
  * High: ₹347.00
  * Low: ₹336.05
  * Close: ₹337.65 ▼ 1.80%
  * Volume: 3,740,535

TECHNICAL INDICATORS:
- 7-Day Moving Average: ₹348.09
- 14-Day RSI: 40.90



In [17]:
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.messages import HumanMessage, AIMessage, SystemMessage




# def generate_response(ticker):
#     chat_history = []
#     financial_statements = get_financial_statements(ticker)
#     news = get_recent_stock_news(ticker)
#     stock_data = get_stock_price(ticker)
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", """
#         This is information about the stock {ticker} - 
#         Financial Statements:{financial_statements}
#         Stock News:{news}
#         Stock open,close and volume for last 5 days:{stock_data}
#         Using this information, you can make an informed decision about investing in this stock.
#         Talk about the stock's financial statements, news and stock data and other factors that you think are important.
#         Recommend me whether to invest in this stock or not at this point.
#         Tell us how you came to this decision using what data points.
#         Explain your result with supporting data.
#         Keep your answer simple to understand for the end user.
#         """),
#         MessagesPlaceholder(variable_name="chat_history"),
#         ("human", "Should i invest in this {ticker}")
#     ])

#     chain = prompt | llm

#     # Generate the first question automatically
#     response = chain.invoke({"ticker": ticker, "chat_history": chat_history, "financial_statements": financial_statements, "news": news, "stock_data": stock_data})
#     response_content = response.content
#     chat_history.append(AIMessage(response.content))
#     return response_content



# # prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")
# # chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
# print(generate_response("RVNL.NS"))

["Why RVNL's Share Price Jumped Back In Green After 4 Days By Benzinga India", 'IRFC & RVNL Share Price Latest Updates and Q3 Results with Market Analysis', 'Stocks in news: LIC, RVNL, BSE, Hindalco, Tata Motors, Eicher Motors', 'Stocks to watch: IndiGo, RVNL, TaMo, Vi, IRB Infrastructure, Jubilant Foodworks', 'RVNL, IRCTC Share, IRFC Bonus News and Q3 Result updates', 'Vodafone Idea, RVNL among 8 likely inclusions in MSCI index in August rejig. Inflows see at $1,239 million', 'RVNL share price suffers a harsh reversal: buy the dip? By Invezz', 'RVNL, NHPC, Reliance, Airtel, HUDCO share news', 'RVNL Shares Continue Recovery After Bagging ₹739 Cr Order By Benzinga India', 'Tata Motors, Cochin Shipyard, RVNL, and VBL Stock Updates: Insights and Future Prospects']
Based on the information provided, here are some key points to consider before making a decision on whether to invest in RVNL.NS:

1. **Financial Statements**:
   - The company's financial statements show a consistent increase i

In [27]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
load_dotenv()
import os
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


    
def stock_analysis(ticker):
    financial_statements = get_financial_statements(ticker)
    news = get_recent_stock_news(ticker)
    stock_data = get_stock_price(ticker)
    stock_info_for_month = get_stock_info_for_month(ticker)

    return financial_statements, news, stock_data, stock_info_for_month



def generate_response_from_stock_info(ticker):
    financial_statements, news, stock_data, stock_info_for_month= stock_analysis(ticker)
    prompt = """
    This is information about the stock {ticker} - 
        Financial Statements:{financial_statements}
        Stock News:{news}
        Stock open,close and volume for last 5 days:{stock_data}
        Stock information for the last month:{stock_info_for_month}
        Using this information, you can make an informed decision about investing in this stock.
        Talk about the stock's financial statements, news and stock data and other factors that you think are important.
        Recommend me whether to invest in this stock or not at this point.
        Tell us how you came to this decision using what data points.
        Explain your result with supporting data.
        Keep your answer simple to understand for the end user 
    """


    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")

    # Build the full prompt with the transcript text
    actual_prompt = ChatPromptTemplate.from_messages([
        ("system", prompt),
        ("human", "{input}")
    ])

    chain = actual_prompt | llm
    response = chain.invoke({"input": "Give me a proper analysis on whether to invest in this stock or not and a little on how is it performing in the market backed by some data. DONOT give me veryy long analysis. Keep your verdict easy to understand even if i have little to no knowledge about the stock", "ticker": ticker, "financial_statements": financial_statements, "news": news, "stock_data": stock_data, "stock_info_for_month": stock_info_for_month})
    
    # Return the notes content
    return response.content


generate_response_from_stock_info("RVNL.NS")




["Why RVNL's Share Price Jumped Back In Green After 4 Days By Benzinga India", 'RVNL share price jumps over 5% on bagging ₹239 crore order from Southern Railway', 'RVNL share price suffers a harsh reversal: buy the dip? By Invezz', 'Stocks to watch: IndiGo, RVNL, TaMo, Vi, IRB Infrastructure, Jubilant Foodworks', 'RVNL Shares Continue Recovery After Bagging ₹739 Cr Order By Benzinga India', 'Lok Sabha Elections 2024: Railway stocks soar on exit polls; IRFC, RVNL among top gainers', 'RVNL Shares Rocket 6% To New Lifetime High After Partnering With DMRC By Benzinga India', 'Chandan Taparia recommendations: How to trade Airtel, Dixon Tech, RVNL shares on May 31', 'Stocks to buy or sell: Cochin Shipyard to RVNL — Sumeet Bagadia recommends five breakout stocks today — May 24', 'Railway Stocks RVNL, IRFC, Railtel Shoot Up After ₹24,657 Cr Projects Announcement By Benzinga India']


"**Stock Analysis: RVNL.NS**\n\n**Financial Health:**\nRVNL's financial statements show a significant increase in total debt from ₹64,413 crore in 2023 to ₹60,325 crore in 2024. However, its net debt has decreased from ₹55,711 crore to ₹48,837 crore. This indicates that the company is trying to reduce its debt burden.\n\n**Market Performance:**\nThe stock has been performing well in the market, with a monthly change of 0.58% and a current price of ₹337.65. The average daily volume is 8,509,036 shares, indicating high liquidity.\n\n**Recent News:**\nThe company has bagged several orders worth ₹239 crore and ₹739 crore, which is a positive sign for the stock.\n\n**Technical Indicators:**\nThe 14-Day RSI (Relative Strength Index) is 40.90, indicating that the stock is not overbought or oversold.\n\n**Verdict:**\nBased on the analysis, I would recommend investing in RVNL.NS at this point. The company's efforts to reduce debt, recent orders, and positive market performance make it an attrac